In [2]:

espectros = ce.recupera_espectros()
X,y = ce.get_x_y(espectros,'pigmento')
lb = LabelEncoder()
y = lb.fit_transform(y)

No se pudo obtener el espectro


In [3]:
from sklearn.calibration import LabelEncoder
import torch
from torch import nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

import control_entrenador as ce

class Estandar(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2151,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 5),
            nn.ReLU(),
            nn.Linear(5, 1),
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
device = "cuda" if not torch.cuda.is_available() else "mps"
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

model = Estandar()
print(model)

Estandar(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2151, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=5, bias=True)
    (5): ReLU()
    (6): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [4]:

X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.long)

train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [8]:
model(X_train)  

tensor([[0.1409],
        [0.1409],
        [0.1409],
        ...,
        [0.1607],
        [0.1672],
        [0.1676]], grad_fn=<AddmmBackward0>)

In [9]:
y

array([74, 74, 74, ..., 69, 45, 47], dtype=int64)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)
print("Done!")

Epoch 1
-------------------------------


IndexError: Target 50 is out of bounds.